<a href="https://colab.research.google.com/github/edgarbc/llm-knowledge-extractor/blob/main/my_RAG_example_PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example of a simple RAG to query an LLM on local data from PDFs.

Edgar Bermudez

November, 2023

Retrieval augmented generation (RAG) provides a way to optimize the output of an LLM with targeted information without modifying the underlying model itself.

This targeted information can be more up-to-date than the LLM and can be specific to a particular organization and industry [REF](https://www.oracle.com/artificial-intelligence/generative-ai/retrieval-augmented-generation-rag/).

The RAG method works by fetching up-to-date or context-specific data from an external database and making it available to an LLM when asking it to generate a response. This approach can help to reduce the likelihood of hallucinations  and improve the accuracy of the LLMs with more up-to-date information.

The key points are :

- how to ingest data so that can be passed to the encoder part of the LLM to produce embeddings so that we can do semantic search on the new data
- map the query to the space with the new embeddings using the same encoder and use the decoder part of the model to generate the response

In [1]:
# install the required modules
!pip install openai
!pip install llama-index
!pip install tiktoken
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.5/943.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing install

In [2]:
import openai
import tiktoken
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import set_global_service_context

Load the API_KEY to be able to use gpt-3.5-turbo LLM from OpenAI.

TODO: extend to an opensource LLM from HuggingFace.

In [3]:
import os

In [4]:
os.environ['OPENAI_API_KEY']="API_KEY"

Load the PDF documents stored in the local directory data_dir. Here we assume that PDFs are uploaded into the data_dir. However, if you want to keep your data in the cloud in a permanent storage like google cloud storage (GCS) you can use unstructured (https://pypi.org/project/unstructured/).

In [5]:
data_dir = "/content/drive/MyDrive/Colab Notebooks/paper_data/"
!ls "/content/drive/MyDrive/Colab Notebooks/paper_data/"

2209.07162.pdf	2302.05543.pdf	2307.15208.pdf


In [6]:
#from llama_index.readers.file.base import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_dir="/content/drive/MyDrive/Colab Notebooks/paper_data/").load_data()

Check that loading the PDFs was successful

In [7]:
documents[1]

Document(id_='e4ffc334-ab9c-4372-a977-c295e6bcc90a', embedding=None, metadata={'page_label': '2', 'file_name': '2209.07162.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/paper_data/2209.07162.pdf', 'file_type': 'application/pdf', 'file_size': 6234415, 'creation_date': '2023-11-07', 'last_modified_date': '2023-11-07', 'last_accessed_date': '2023-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='f1935e7d9d77101e716dec79166ad2d24671a696ce9902cfa006b83dcc4a8a6e', text='2 Pinaya et al.\nsentences [8,26]. During the same period, medical image analysis also made re-\nmarkable breakthroughs by applying deep neural networks to solve tasks such\nas segmentation, structure detection, and computer-aided diagnosis (detailed re-\nview available at [

Now we need to split the text into tokens. We can use tiktoken for this and openAI gpt to produce the encodings.

In [9]:
# you can parse the documents into nodes using the default values
text_splitter = TokenTextSplitter(
    separator=" ",
    chunk_size=1024,
    chunk_overlap=20,
    backup_separators=["\n"],
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

node_parser = SimpleNodeParser.from_defaults()

Or you can be more specific to define how to parse the documents. For example:

In [ ]:
from llama_index.node_parser import SentenceSplitter
text_splitter=SentenceSplitter(
    separator=" ",
    chunk_size=1024,
    chunk_overlap=20,
    paragraph_separator="\n\n\n",
    secondary_chunking_regex="[^,.; ]+[,.; ]?",
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)



Now it is time to get the embeddings using the GPT to do the encoding.

In [10]:
from llama_index.embeddings.openai import OpenAIEmbedding
llm = OpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=256)
embed_model = OpenAIEmbedding()
prompt_helper = PromptHelper(
    context_window=4096,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None
)
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
    prompt_helper=prompt_helper
)

Now we index the documents with the context

In [11]:
from llama_index.indices.vector_store.base import VectorStoreIndex
index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context
)

Now define how we are going to query the knowledge (vector) database

In [12]:
query_engine=index.as_query_engine(service_context=service_context)


Now let's ask something specific that is present in the documents we augmented it with:

In [13]:
response = query_engine.query("Who proposed a 3D GAN with a hierarchical structure?")
print(response)

Sun et al. proposed a 3D GAN with a hierarchical structure.
